# FIT5196 Assessment 1 Task 2 Extract Data from a PDF File

#### Student Name: Vipul Krishnan M.D.
#### Student ID: 28104641

Date: 02/04/2018

Version: 1.0

Environment: Python 3.6.0 and Anaconda 4.3.0 (64-bit)

Libraries used:
* pandas (version 0.20.1)
* pdftables (version 0.0.5) (Need only get_tables function)
* re (version 2.2.1)

External Files:
* pdf2txt.py

## 1. Introduction
The PDF file, "health.pdf", contains the children's health data over 202 countries in the world. The table spreads over four pages. The task is to extract the table and save them in a CSV file. In order to finish the task

* The tables has to be correctly parsed
* If the number followed by a character "x" in the pdf file,  "x" must be dropped in your script
* The extracted table should be saved in a CSV file named as "health.csv"
* The input file must only be "health.pdf".

## 2. Import Libraries

In [ ]:
from pdftables import get_tables
import re
import pandas as pd

## 3. Importing Data from PDF file

We import data using two different ways. The first one is using the library- pdftables (Hopkinson I., 2013), and the second one is using an external file pdf pdf2text.py (PDFMiner, 2016), which comes with the pdfminer package (PDFMiner, 2016).

Although pdftables obtains all the values in a table format, some of the rows may be merged and we have to sperate manually. Sometimes we may not be able to find where to split exactly. For example, if there is a value 234 and we know that it is a merge between two numbers, it can be 23,4 or 2,34. In those cases we use the text obtained from the pdf2text.py as a reference.

### 3.1 Using pdftables 

In [ ]:
pdfFile = 'health.pdf' # declaring input file name
pdfobj = open(pdfFile, 'rb') # creating file object 
tables = get_tables(pdfobj) # extracting tables from the file object using the function from pdftables

To get the priliminary idea of the data, we can just print all the extracted tables

In [ ]:
for table in tables: # for each extracted table
    for row in table: # for rach row in the table
        print(row) # print the row
    print('==========================\n') # table seperator

### 3.2 Using pdf2text.py

We can use pdf2txt.py as below (UnixUser.org, 2014)

In [ ]:
# using the external file pdf2txt.py for extracting text from health.pdf to a file tables.txt
!pdf2txt.py -o tables.txt health.pdf

Now we copy the text line by line from the file and store as a list for searching in it easily

In [ ]:
pdfTxtFile = './tables.txt'
pdf_txt = open(pdfTxtFile, 'r') # openinng the file in read mode

linelist = [] # creating an empty list

# loop over all the lines...
for line in pdf_txt:
    linelist.append(repr(line)) # and add each line to list

print(linelist)

The data is not clean and not useful for any extraction purpose unless it is cleaned. But as we just need to confirm the values, we dont need to spend time for cleaning it.

## 4. Data Analysis and Extraction
From the above printed data, we can see that for each table, the required portion extends at most from 6th row to second last row. (The last table is an exception. We can extract it later). So we can print rows omly from index 5 to -2

In [ ]:
for table in tables: # for each extracted table
    for row in table[5:-2]: # for rach row from 6th to second last in the table
        print(row) # print the row
    print('==========================\n') # table seperator

### 4.1 Extracting Tables
Looking at the table-1, the only major peoblem is the merged 5th and 6th columns (column index 4 and 5). We need some code to seperate these columns. As there are similar stuations in the other tables also, we can write this code as a function for reusability. PFB the function. The details are given below the function

In [ ]:
# Helper function for seperating merged 2 columns
# Only applicable for tables with only two rows merged together and rest all good
def helperfun1(row, cnum):
    # arguments 1. The actual row in the extracted table, 2. The index of the first merged column
    
    new_row = [] # creating an empty row for result
    
    for j in range(len(row)):
        if j == cnum: #act only for the merged column. else just add the existing values (else condition at bottom)
            
            if (len(row[j]) == 4) & (row[j][:3] != '100') & (row[j][-3:] != '100'): # if the merged length is 4 and no 100
                # then just split the 4 digit value to 2 two digit values
                new_row.append(row[j][:2])
                new_row.append(row[j][2:])
                
            elif (len(row[j]) == 4) & (row[j][:3] == '100'): # if 4 digits with a leading 100
                new_row.append(row[j][:2]) # add the 100 first
                new_row.append(row[j][-1]) # add the last digit as a seperate value
                
            elif (len(row[j]) == 4) & (row[j][-3:] == '100'): # if 4 digits with a trailing 100
                new_row.append(row[j][1]) # add the first digit as a single value
                new_row.append(row[j][-3:]) # then add the 100
                
            elif len(row[j]) == 6: # if 6 digits. This happens only in the case of 100100
                # add two 100s
                new_row.append(100)
                new_row.append(100)
                
            elif (len(row[j]) == 3) and ('–' in row[j]): # 3 digit with a -
                if row[j][0] == '–': # - is at first
                    new_row.append('–') # add - first
                    new_row.append(row[j][1:]) # add last two digits as a single value 
                else: # - is at last 
                    new_row.append(row[j][:2]) # add the first two digits as a single number
                    new_row.append('–') # add -
                    
            elif (len(row[j]) == 3): # if the length is 3 without -
                
                # The below code splits the 3 digits into two values by making the difference between them least. 
                # Assumes there exists some correlation between these values and this assumption seems to be holding good
                # The values obtained is tested using the pdftotext.py approach and it holds good
                
                if (int(row[j][1:]) - int(row[j][0])) < (int(row[j][:2]) - int(row[j][-1])):
                    new_row.append(row[j][0])
                    new_row.append(row[j][1:])
                
                else:
                    new_row.append(row[j][:2])
                    new_row.append(row[j][2:])
                    
            elif (len(row[j]) == 5): # five digits. Possible when there is a 100 and two digit number
                
                if row[j][0:3] == '100': # if there is leading 100 
                    new_row.append('100')
                    new_row.append(row[j][3:])
                else:
                    new_row.append(row[j][:2]) # if there is trailing 100
                    new_row.append('100')
                    
            elif (len(row[j]) == 2): # if only 2 digits, just split into 2 seperate numbers and add
                new_row.append(row[j][:1])
                new_row.append(row[j][1:])
        else: #else just add the existing values
            new_row.append(row[j])
            
    return new_row # return the new row

The above function is a helper function to seperate merged row pairs. We are going to apply this function in multiple situations.

The function arguments are
1. The actual row as a list
2. The position of merged row in the actual row

The function simply adds all rows except the merged rorw directly to the new row (new_row variable).

More the merged row, it cahcks the following 'if' conditions and acts according to it.

* if the value has 4 digits and there is no 100 in it, it splits the value into two halves.
* if the value has 4 digits and there is a trailing 100, it takes the first digit as a single number and 100 as second
* if the value has 4 digits and there is a leading 100, it takes the last digit as a single number and 100 as the first number
* if the length is 6, then the numbers are two 100s
* if the length is 3 and there is a - in it, it sperates the - and take the other digits as a single value
* if it is 3 digits without a -, then we use a correlative approach. it splits the number into two so that the difference between them is least
* if there is 5 digits, it checks for leading 100 or trailing 100 and seperates according to it
* if it is just two digits, just make it two halves.


Now we apply this to table 1, and we use pandas (pandas, 2017) for converting the table in list format to a dataframe (Williems k., 2018).

In [ ]:
table1 = [] # initialising an empty table1 list
for row in tables[0][5:-2]:  
    table1.append(helperfun1(row, 4)) # creating the corrected rows in table1 using the helperfun1 function (Here column 4 is merged)
table1f = pd.DataFrame(table1) # converting it to a pandas daraframe

In [ ]:
table1f[:5] # seeing the data frame

Doing the same for the table 2 and 3. But the merged column can be different

In [ ]:
table2 = [] # initialising empty row
for row in tables[1][5:-2]: # for each row from 5th to second last
    table2.append(helperfun1(row, 5)) # here the merged row is 5th
table2f = pd.DataFrame(table2) # converting to dataframe

In [ ]:
table2f[:5] # seeing the data frame

In [ ]:
table3 = [] # initialising empty row
for row in tables[2][5:-2]: # for each row from 5th to second last
    table3.append(helperfun1(row, 4)) # here the merged row is 4th
table3f = pd.DataFrame(table3) # converting to dataframe

In [ ]:
table3f[:5] # seeing the data frame

The fourth table has a lot of issues. We can check it again.

In [ ]:
for row in tables[3][5:19]: # selecting only the rows containing required parts
    print(row)

As we see, 

* the country name is merged with 3 other columns.
* the next number is a combination of 6 columns
* columns 15, 16, 17 are merged together
* columns 19 and 20 are merged together

For seperating the merged rows, we need two functions. One for merged 3 rows and one for merged 2 rows. As we have to define function before our main code, writing the functions below. These will be used later in the main code.

In [ ]:
# This function splits merged 2 numbers
# Although it doesnt cover all the use cases in a general scenario, it covers everything in our example.
def helperfun2(ele, new_row):
    
    if '–' in ele: # if there is a -  * In all our data, there is only trailing -. so only this case is considered
        new_row.append(ele.strip('–')) # ass the '-' removed part first
        new_row.append('–') # and then add -
    else:
        new_row.append(ele[:len(ele)//2]) # In all other conditions, just split into two halves. # this holds good for our data
        new_row.append(ele[len(ele)//2:])
        
    return new_row # return the new row



This function is for spliting 2 merged values.It takes a merged value and the row list two which the values to be added

* if there is a '-' in the value, it put the number part first and then places a -. If we llok at our data, we can see that this holds good for all our values
* else it simply splits the data equally into 2 halves.

In [ ]:
# This function splits merged 3 numbers.
# Although it doesnt cover all the use cases in a general scenario, it covers everything in our example.
def helperfun3(num, new_row):
    
    # The below code splits 3 two digit numbers. Although this is generally not correct, it is useable for our case
    if len(num) == 6: # if there are 6 digits in the number
        new_row.append(num[0:2]) # splits each two digit numbers 
        new_row.append(num[2:4]) 
        new_row.append(num[4:6])
        
    elif len(num) == 9: #if there are 9 digits, the only possible case is 100,100,100
        for i in range(3): # add 3 100s
            new_row.append(100)
            
    elif len(num) == 7: # if there are 7 digits, it should contain a 100. Also we dont have any with 2 and only 2 100s
        
        if num[0:3] == '100': #if the 100 is first
            new_row.append(100) 
            new_row.append(num[3:5])
            new_row.append(num[5:7])
            
        elif num[-3:] == '100': #if the 100 is last
            new_row.append(num[0:2])
            new_row.append(num[2:4])
            new_row.append(100)
        
        else: # if the 100 is at middle
            li = num.split('100')
            new_row.append(li[0])
            new_row.append("100")
            new_row.append(li[1])
            
    elif '–' in num: #if it contains a hash sign
        
        # visibily there is only one row which contains a dash, so the function is written for that row
        new_row.append('–')
        new_row.append(num.strip('–'))
        new_row.append('–')
        
    return new_row # returning the new row

The above function is for splitting 3 merged numbers.

It accepts the merged number and the new row list

* if the length of the number is 6, then it is split into 3 two digit numbers. Although this is generally not correct, it is useable for our case
* if the length is 9, then 3 100s are added to the new row
* if there are 7 digits, it should contain a 100. (Seeing the data, we dont have any with 2 and only 2 100s). so we just find the position of 100 first. The remaining can be split into two 2 digit numbers.
* if the value contains a '-' : visibily there is only one case with situation for which we we have to put the number in middle column and put the '-' both after and before it


Note that these functions are used on multiple occassions in the main code (code resusing). The main code is given below.

In [ ]:
table4 = [] # variable for table in the 4th page

for row in tables[-1][5:19]: #for each rows in last table
    new_row = [] #for current row 
    for j in range(len(row)): # for each elements in the row

        if j == 0: # if it is first element
            new_row.append(re.findall(r'[^-x\d–]+', row[j])[0]) # seperating country name with re
            num = re.findall(r'[\d–]+', row[j]) # adds the country name path as first element of the row
            new_row = helperfun3(num[0], new_row) # calls the helper funtion (same function is used at different places)
            
        elif j == 1: # if j is the second element of the read row
            
            if '–' in row[j]: # if there is a '-'. This is an exceptional case
                first = row[j][:-5] # the first merged 3 numbebrs are seperated...
                new_row = helperfun3(first, new_row) # using helper fun
                new_row.append('–') # then the - is added.
                new_row.append(row[j][-4:-2]) # all last three values in all has 2 digits. using this pattern, last values are seperated
                new_row.append(row[j][-2:])
            else:
                first = row[j][:-6] # except the exception case, all folllows a pattern that, last 3 merged values are 2 digits. Using this fisr part is seperated.. 
                new_row = helperfun3(first, new_row) # ... and first part is partitioned into correct 3 numbers using helper fun
                new_row.append(row[j][-6:-4]) # The last 3 numbers are ll 2 digits. So the 6 digits are seperated into 3 numbers
                new_row.append(row[j][-4:-2])
                new_row.append(row[j][-2:])
                
        elif j in [2, 3, 4, 5, 6, 8, 10, 11]: # All these column numbers are correctly parsed by the pdftable. So no need to make any change
            new_row.append(row[j])
         
        # the column 8 (index 7) contains 3 merged values. But it is very different to the similar other scenarios. So we have to write seperate code
        # visibily there is no 100s in the merged values
        elif j == 7:
            if len(row[j]) == 3: # if it is only 3 digits, just make three different numbers
                new_row.append(row[j][0])
                new_row.append(row[j][1])
                new_row.append(row[j][2])
                
            elif row[j][0] == '0': # if there is a leading 0, sperate it first and seperate the rest using helperfun2
                    new_row.append(0)
                    new_row = helperfun2(row[j][1:], new_row)
                    
            elif ('–' in row[j]) and (len(row[j]) == 5): # if the length is 5 and contains '-' 
                new_row.append(row[j][0:2])
                new_row.append(row[j][2:4])
                new_row.append('–') # this case happens only in one exceptional case and is visible. So we dont need to try all cases to find the position of -
            
            elif len(row[j]) == 6: # if the length is 6, as there is no 100, simply split into 3 two digit numbers
                new_row.append(row[j][0:2])
                new_row.append(row[j][2:4])
                new_row.append(row[j][4:6])
                
            # this is an exceptional case. We dont have any way to predict how the number is. So we use data from obtained using pdf2txt.py method,
            # and we can manually find the splits. 
            # the process is shown below
            # The split is as dd,d,dd. So we made the below code accordingly
            elif len(row[j]) == 5:
                new_row.append(row[j][0:2])
                new_row.append(row[j][2:3])
                new_row.append(row[j][3:5])
            
        elif j == 9: # this is a merge between two numbers
            # there is no - number and no 100
            new_row = helperfun2(row[j], new_row) # using helperfun2 for extraction
    table4.append(new_row)

In the above code, the required part of the last table is extracted first, ie only the rows with country names.

In each row, each column is analysed. 

As the first column is a merge between the country name and the three succedeing values:
    The country name is extracted first using python re.
    The remaining three digits can be sepetated and added using helperfun3 defined above

The second column is a merge between 6 numbers. All the last 3 numbers are 2 digits, except in one case which contains a '-'.       The value containing '-' is porcessed as an exceptional case first. 
    For other values, the value is spit into 2 halves first as 3 values in each half and then sperated thesese halves using the     helperfun3
    Since all the last three values are 2 digits, we can easily seperate them

The columns with indices 2, 3, 4, 5, 6, 8, 10, 11 are parsed correctly. So they dont need any editing

The j =9 case can be extracted using helperfun2.

The column 8 (index 7) contains 3 merged values. But it is very different to the similar other scenarios. So we have to write seperate code. Visibily there is no 100s in the merged values.
    If it is only three digit, we just split them as 3 different numbers
    If the first value is a 0, we make it as a number and split rest using helperfun2
    If there are 5 digits and there is a -, it happens only in one case and in that case the '-' is at the first place. So we split the other part equally into 2 numbers
    If there are 6 digits, and as there is no 100, we simply split into 3 two digit numbers.
    Now there is one exceptional case in which we have 5 digits and we dont have any clue about how to split it. So we use the text extracted using pdf2txt.py as below
    
    

The easiest way to find the position of the required number is to search using a country name near it

In [ ]:
# searching for Uganda
for a in linelist:
    if "Uganda" in a:
        print(linelist.index(a))

So it should be somewhere after 4622. Using the trial and error method we can easily get the below part containing the required information. Also we know all the surrounding number of these numbers.

In [ ]:
linelist[4932:4948]

Comparing the above data with our table, we can see that this is the 17th column (index 16). And we can see a 7 there in our required position. So we can reach the conclusion that we have to split 7 as a seperate number, which means the split will be as (dd, d, dd). We write the code manualy to split it accordingly.

Now we have completed the 4th table. We make it a DataFrame

In [ ]:
table4f = pd.DataFrame(table4) # converting to DataFrame

In [ ]:
table4f # printing

### 4.2 Combining Tables to Form the Complete Table

Now we can concatenate all the datafrmaes to form the complete table (Williems, 2018)

In [ ]:
frames = [table1f, table2f, table3f, table4f ] # creating an list of all the table dataframes
combined = pd.concat(frames) # using pandas concat method to form the combined dataframe

We can just see the first few lines

In [ ]:
# showing first 5 lines of the dataframe
combined[:5]

The name of the columns has to be changed as [Country Name,0,1,....,21]. For this we can use the below code

In [ ]:
# renaming the columns
combined.columns = ['Country Name'] + list(range(0,22))

As shown above, the dataframe contains '-' and trailing x with some values. We have to remove them. As there is no country names starting or ending with 'x' or '-', we can simply strip these from all values to remove them. Also we can strip any spaces.

In [ ]:
# converting all values in dataframe to string, stripping 'x', '-', and blank spaces
combined =combined.applymap(str).applymap(lambda x: x.strip('x')).applymap(lambda x: x.strip('–')).applymap(lambda x: x.strip(' '))

Aslo we have to reset the row index.

In [ ]:
combined.reset_index(drop=True, inplace=True) # resetting row index

Now we can just print it to see

In [ ]:
print(combined) # printing the final dataframe

## 5 Writing to File
Now writing the file to health.csv file

In [ ]:
combined.to_csv("health.csv", index=False) # writing to file

## 6 Summary

In order to extract the data from a pdf file we used 2 methods, using pdftables and using an external file pdf2txt.py. The second method is just for reference in case og ambiguities.

There where 4 tables as there were 4 pages in the document. We cleaned the problems such as merged rows and unnecessary symbnols such as '-' and 'x' sing python code and then we combined all 4 tables. The pandas library is used for various purposes throughout the process.

The combined table is then saved as a csv file.

## 7 Referneces

- Hopkinson I. (2013, July 29). *pdftables – a Python library for getting tables out of PDF files*. Retrieved from https://scraperwiki.com/2013/07/pdftables-a-python-library-for-getting-tables-out-of-pdf-files/
- PDFMiner. (2016). Retrieved from https://github.com/euske/pdfminer
- Pandas. (2017). Retrieved from https://pandas.pydata.org/
- Williems K. (2018, February 23) *Pandas Tutorial: DataFrames in Python*. Retrieved from https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python
- UnixUser.org (2014, March 24). *PDFMiner*. Retrieved from https://www.unixuser.org/~euske/python/pdfminer/